In [ ]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

In [ ]:
columns=['target','ids','date','flag','user','text']
df=pd.read_csv('twitter.csv',encoding='ISO-8859-1',names=columns)
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
dataset=df[['text','target']]
dataset.head(1)

,text,target
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0


In [ ]:
dataset['target'].unique()

array([0, 4])

In [ ]:
dataset['target']=dataset['target'].replace(4,1)

In [ ]:
dataset['target'].unique()

array([0, 1])

In [ ]:
dataset.text[0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

#Step 1: Remove URL

In [ ]:
pattern=re.compile(r'https?://\S+|www\.\S+')
pattern.sub(r'','Twitter Sentiment Analysis Data set available at https://www.kaggle.com/datasets/kazanova/sentiment140')

'Twitter Sentiment Analysis Data set available at '

In [ ]:
def remove_url(text):
  pattern=re.compile(r'https?://\S+|www\.\S+')
  return pattern.sub(r'',text)

In [ ]:
dataset['text']=dataset['text'].apply(lambda x: remove_url(x))

In [ ]:
dataset['text'][0]

"@switchfoot  - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

#Step 2: Remove HTML Tags

In [ ]:
def remove_tags(text):
  pattern=re.compile(r'<.*?>')
  return pattern.sub(r'',text)

In [ ]:
dataset['text']=dataset['text'].apply(lambda x: remove_tags(x))

In [ ]:
dataset['text'][0]

"@switchfoot  - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

#Step 3: Handling Emojis

In [ ]:
#Emojis
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink',';D': 'wink',':-E': 'vampire', ':(': 'sad',
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed',
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink',
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}


In [ ]:
def handle_emoji(text):
  for emoji in emojis.keys():
    text=text.replace(emoji,"Emoji"+emojis[emoji])
  return text

In [ ]:
handle_emoji("@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D")

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. Emojiwink"

In [ ]:
dataset['text']=dataset['text'].apply(lambda x: handle_emoji(x))

In [ ]:
dataset['text'][0]

"@switchfoot  - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. Emojiwink"

#Step 4: User Handling

In [ ]:
def user_handle(text):
  pattern=re.compile(r'@[^\s]+')
  text=pattern.sub("TUser",text)
  return text
dataset['text']=dataset['text'].apply(lambda x: user_handle(x))

#Step 5: Punctuation Removal

In [ ]:
import string

In [ ]:
punc=string.punctuation

In [ ]:
def remove_punc(text):
  return text.translate(str.maketrans("","",punc))

In [ ]:
remove_punc('LOL! It was very funny. ROFL' )

'LOL It was very funny ROFL'

In [ ]:
dataset['text']=dataset['text'].apply(lambda x: remove_punc(x))

#Step 6: Chat Word Treatment

In [ ]:
slang="/content/slang.txt"

In [ ]:
slang

'/content/slang.txt'

In [ ]:
with open(slang,'r') as f:
  lines=f.readlines()

In [ ]:
lines

['AFAIK=As Far As I Know\n',
 'AFK=Away From Keyboard\n',
 'ASAP=As Soon As Possible\n',
 'ATK=At The Keyboard\n',
 'ATM=At The Moment\n',
 'A3=Anytime, Anywhere, Anyplace\n',
 'BAK=Back At Keyboard\n',
 'BBL=Be Back Later\n',
 'BBS=Be Back Soon\n',
 'BFN=Bye For Now\n',
 'B4N=Bye For Now\n',
 'BRB=Be Right Back\n',
 'BRT=Be Right There\n',
 'BTW=By The Way\n',
 'B4=Before\n',
 'B4N=Bye For Now\n',
 'CU=See You\n',
 'CUL8R=See You Later\n',
 'CYA=See You\n',
 'FAQ=Frequently Asked Questions\n',
 'FC=Fingers Crossed\n',
 "FWIW=For What It's Worth\n",
 'FYI=For Your Information\n',
 'GAL=Get A Life\n',
 'GG=Good Game\n',
 'GN=Good Night\n',
 'GMTA=Great Minds Think Alike\n',
 'GR8=Great!\n',
 'G9=Genius\n',
 'IC=I See\n',
 'ICQ=I Seek you (also a chat program)\n',
 'ILU=ILU: I Love You\n',
 'IMHO=In My Honest/Humble Opinion\n',
 'IMO=In My Opinion\n',
 'IOW=In Other Words\n',
 'IRL=In Real Life\n',
 'KISS=Keep It Simple, Stupid\n',
 'LDR=Long Distance Relationship\n',
 'LMAO=Laugh My A..

In [ ]:
lines[0].split('=')[1][:-1]

'As Far As I Know'

In [ ]:
lines[0].split("=")[0]

'AFAIK'

In [ ]:
chat_words=dict()
for i in range(len(lines)):
  chat_words[(lines[i].split('='))[0]]=(lines[i].split('='))[1][:-1]

In [ ]:
chat_words

{'AFAIK': 'As Far As I Know',
 'AFK': 'Away From Keyboard',
 'ASAP': 'As Soon As Possible',
 'ATK': 'At The Keyboard',
 'ATM': 'At The Moment',
 'A3': 'Anytime, Anywhere, Anyplace',
 'BAK': 'Back At Keyboard',
 'BBL': 'Be Back Later',
 'BBS': 'Be Back Soon',
 'BFN': 'Bye For Now',
 'B4N': 'Bye For Now',
 'BRB': 'Be Right Back',
 'BRT': 'Be Right There',
 'BTW': 'By The Way',
 'B4': 'Before',
 'CU': 'See You',
 'CUL8R': 'See You Later',
 'CYA': 'See You',
 'FAQ': 'Frequently Asked Questions',
 'FC': 'Fingers Crossed',
 'FWIW': "For What It's Worth",
 'FYI': 'For Your Information',
 'GAL': 'Get A Life',
 'GG': 'Good Game',
 'GN': 'Good Night',
 'GMTA': 'Great Minds Think Alike',
 'GR8': 'Great!',
 'G9': 'Genius',
 'IC': 'I See',
 'ICQ': 'I Seek you (also a chat program)',
 'ILU': 'ILU: I Love You',
 'IMHO': 'In My Honest/Humble Opinion',
 'IMO': 'In My Opinion',
 'IOW': 'In Other Words',
 'IRL': 'In Real Life',
 'KISS': 'Keep It Simple, Stupid',
 'LDR': 'Long Distance Relationship',
 'LM

In [ ]:
def chat_word_treatment(text):
  new_text=[]
  for w in text.split():
    if w in chat_words:
      new_text.append(chat_words[w])
    else:
      new_text.append(w)
  return " ".join(new_text)

In [ ]:
chat_word_treatment(remove_punc("LOL! It was very funny. ROFL"))

'Laughing out loud It was very funny Rolling On The Floor Laughing'

In [ ]:
dataset['text']=dataset['text'].apply(lambda x: chat_word_treatment(x))

In [ ]:
dataset['text'][0]

'TUser Awww thats a bummer You shoulda got David Carr of Third Day to do it Emojiwink'

#Step 7: Make Lower Case

In [ ]:
dataset['text']=dataset['text'].str.lower()

#Step 8: Spelling Correction

In [ ]:
from textblob import TextBlob

In [ ]:
inc_Text="Helllo ! How are you?"

In [ ]:
str(TextBlob(inc_Text).correct())

'Hello ! Now are you?'

In [ ]:
def spelling_correct(text):
  return str(TextBlob(inc_Text).correct())

In [ ]:
#dataset['text']=dataset['text'].apply(lambda x: spelling_correct(x))

#Step 9: Tokenization

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def word_token(text):
  return word_tokenize(text)

In [ ]:
dataset['text']=dataset['text'].apply(lambda x: word_token(x))

In [ ]:
word_token("I love pizza")

['I', 'love', 'pizza']

#Step 10: Stop Word Removal

In [ ]:
text1=word_token(("I am loving pizza").lower())

In [ ]:
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
st_w=stopwords.words('english')

In [ ]:
clean_text=[word for word in text1 if word not in st_w]

In [ ]:
clean_text

['loving', 'pizza']

In [ ]:
" ".join(clean_text)

'loving pizza'

#Step 11: Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer
st=PorterStemmer()

In [ ]:
st.stem("caring")

'care'

In [ ]:
def stem(text):
  text=[st.stem(word) for word in text]
  return " ".join(text)

In [ ]:
#dataset['text']=dataset['text'].apply(lambda x: stem(x))